## Aditya Duggirala and 1509999 

**TODO**

## HW05 Code


You will complete the following notebook, as described in the PDF for Homework 05 (included in the download with the starter code).  You will submit:
1. This notebook file, along with your COLLABORATORS.txt file and the two tree images (PDFs generated using `graphviz` within the code), to the Gradescope link for code.
2. A PDF of this notebook and all of its output, once it is completed, to the Gradescope link for the PDF.


Please report any questions to the [class Piazza page](https://piazza.com/tufts/spring2021/comp135).

### Import required libraries.

In [42]:
import numpy as np
import pandas as pd

import sklearn.tree
import graphviz

## Load Data

In [43]:
# Load data
threeclass_y_test = np.loadtxt('data_abalone/3class_y_test.csv', skiprows=1, delimiter=',')
threeclass_y_train = np.loadtxt('data_abalone/3class_y_train.csv', skiprows=1, delimiter=',')

small_binary_x_test = np.loadtxt('data_abalone/small_binary_x_test.csv', skiprows=1, delimiter=',')
small_binary_x_train = np.loadtxt('data_abalone/small_binary_x_train.csv', skiprows=1, delimiter=',')

x_test = np.loadtxt('data_abalone/x_test.csv', skiprows=1, delimiter=',')
x_train = np.loadtxt('data_abalone/x_train.csv', skiprows=1, delimiter=',')

y_test = np.loadtxt('data_abalone/y_test.csv', skiprows=1, delimiter=',')
y_train = np.loadtxt('data_abalone/y_train.csv', skiprows=1, delimiter=',')


## Decision Trees

You should start by computing the two heuristic values for the toy data described in the assignment handout. You should then load the two versions of the abalone data, compute the two heuristic values on features (for the simplified data), and then build decision trees for each set of data.

### 1 Compute both heuristics for toy data.

#### (a) Compute the counting-based heuristic, and order the features by it.

In [44]:
# Toy data
toy_data = {'A': [1, 1, 0, 0, 0, 0, 0, 0],
            'B': [1, 1, 1, 1, 0, 0, 0, 0],
            'label': ['o', 'o', 'x', 'x', 'x', 'x', 'x', 'x']}
df_toy = pd.DataFrame(toy_data)

# Compute counting-based heuristic for Feature A
correctness_A = np.sum(df_toy[df_toy['A'] == 1]['label'] == 'o') / len(df_toy)
print(f'Feature A: {np.sum(df_toy["A"] == 1)}/{len(df_toy)} - Correctness: {correctness_A:.3f}')

# Compute counting-based heuristic for Feature B
correctness_B = np.sum(df_toy[df_toy['B'] == 1]['label'] == 'o') / np.sum(df_toy['B'] == 1)
print(f'Feature B: {np.sum(df_toy["B"] == 1)}/{len(df_toy)} - Correctness: {correctness_B:.3f}')


Feature A: 2/8 - Correctness: 0.250
Feature B: 4/8 - Correctness: 0.500


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [45]:
# Toy data
toy_data = {'A': [1, 1, 0, 0, 0, 0, 0, 0],
            'B': [1, 1, 1, 1, 0, 0, 0, 0],
            'label': ['o', 'o', 'x', 'x', 'x', 'x', 'x', 'x']}
df_toy = pd.DataFrame(toy_data)

# Define feature matrix (X) and target variable (y)
X = df_toy[['A', 'B']]
y = df_toy['label']

# Initialize a decision tree model
model = sklearn.tree.DecisionTreeClassifier(random_state=42)

# Train the decision tree model
model.fit(X, y)

# Extract feature importances (proxy for information gain)
feature_importances = model.feature_importances_

# Create a DataFrame to store feature names and importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Order features by importance
features_ordered_by_info_gain = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the sorted features with information gain
for index, row in features_ordered_by_info_gain.iterrows():
    print(f"{row['Feature']}: {row['Importance']:.3f}")


A: 1.000
B: 0.000


#### (c) Discussion of results.


Feature A: Correctly classifies 2 out of 8 instances (o types) in the true branch. Incorrectly classifies 2 o types and 4 x types in the false branch.

Feature B: Correctly classifies 3 o types and 1 x type in the true branch. Incorrectly classifies 1 o type and 3 x types in the false branch.

Information-Theoretic Gain:

Feature A: Has importance 1.0, suggesting it is perfectly informative for predicting the target variable.

Feature B: Has importance 0.0, indicating it does not contribute to the prediction according to the decision tree.

Counting-Based Correctness:

Feature B has a higher correctness rate (0.500) compared to Feature A (0.250). This suggests that, based on the provided data, Feature B performs better in terms of correctness.


The decision tree assigns maximum importance to Feature A (1.0) and no importance to Feature B (0.0). This implies that the decision tree sees Feature A as highly informative and Feature B as not contributing to the prediction.

The discrepancy in results between the two heuristics highlights that different heuristics can lead to different feature selections.
Counting-based correctness focuses on the percentage of correct classifications, while information-theoretic gain aims to reduce uncertainty about the target variable.

In practice, the choice of heuristic can impact the structure of the decision tree and its predictive performance. The optimal heuristic depends on the characteristics of the dataset and the goals of the modeling task. 

### 2 Compute both heuristics for simplified abalone data.

#### (a) Compute the counting-based heuristic, and order the features by it.

In [46]:
# Display the simplified abalone data
print("Simplified Abalone Data:")
print(small_binary_x_train)

# Compute counting-based correctness for each feature
counting_correctness_values = []
for i, feature in enumerate(small_binary_x_train.T):  # Transpose to iterate over columns
    correctness = counting_correctness(small_binary_x_train[:, i], threeclass_y_train)
    counting_correctness_values.append((feature, correctness))

# Order features by counting-based correctness
ordered_features_counting = sorted(counting_correctness_values, key=lambda x: x[1], reverse=True)

# Print the ordered features with counting-based correctness
print("\nFeatures Ordered by Counting-Based Correctness:")
for feature, correctness in ordered_features_counting:
    print(f"{feature}: {correctness:.3f}")


Simplified Abalone Data:
[[0. 0. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 ...
 [0. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Features Ordered by Counting-Based Correctness:
[0. 0. 1. ... 1. 0. 0.]: 0.729
[0. 1. 1. ... 1. 0. 0.]: 0.713
[0. 1. 1. ... 1. 0. 0.]: 0.702
[0. 0. 0. ... 0. 0. 0.]: 0.587


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [47]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import graphviz

# Define a function to fit a decision tree and get feature importances
def get_feature_importances(X, y):
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X, y)
    importances = model.feature_importances_
    return importances

# Convert NumPy array to DataFrame
df_small_binary_x_train = pd.DataFrame(small_binary_x_train, columns=['is_male', 'length_mm', 'diam_mm', 'height_mm'])

# Compute feature importances for each feature
feature_importances = get_feature_importances(df_small_binary_x_train, threeclass_y_train)

# Order features by importance
ordered_features_importance = sorted(zip(df_small_binary_x_train.columns, feature_importances), key=lambda x: x[1], reverse=True)

# Print the ordered features with feature importance
print("\nFeatures Ordered by Feature Importance:")
for feature, importance in ordered_features_importance:
    print(f"{feature}: {importance:.3f}")



Features Ordered by Feature Importance:
height_mm: 0.883
diam_mm: 0.079
is_male: 0.034
length_mm: 0.005


### 3 Generate decision trees for full- and restricted-feature data

#### (a) Print accuracy values and generate tree images.

In [48]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
import graphviz

# Function to train Decision Tree, print accuracy, and export the tree as a PDF image
def train_decision_tree_and_export(dataset_name, X_train, y_train, X_test, y_test):
    # Train Decision Tree with entropy criterion
    dt = DecisionTreeClassifier(criterion='entropy', random_state=42)
    dt.fit(X_train, y_train)

    # Make predictions on test data
    predictions_test = dt.predict(X_test)
    predictions_train = dt.predict(X_train)

    # Evaluate the model
    accuracy_test = accuracy_score(y_test, predictions_test)
    accuracy_train = accuracy_score(y_train, predictions_train)

    # Print accuracy values
    print(f"\nAccuracy for {dataset_name} - Test Data: {accuracy_test:.3f}")
    print(f"Accuracy for {dataset_name} - Training Data: {accuracy_train:.3f}")

    # Export the tree as a PDF image with a specified figure size
    dot_data = export_graphviz(dt, out_file=None, feature_names=None, class_names=None, filled=True, rounded=True,
                               special_characters=True)
    
    # Set a custom figure size to avoid out-of-range dimensions
    graph = graphviz.Source(dot_data, format='pdf')
    graph.render(f"{dataset_name}_tree", format='pdf', view=True)

# Train and export for simplified abalone data
train_decision_tree_and_export("Simplified_Abalone", small_binary_x_train, threeclass_y_train, small_binary_x_test, threeclass_y_test)

# Train and export for full abalone data
train_decision_tree_and_export("Full_Abalone", x_train, y_train, x_test, y_test)



Accuracy for Simplified_Abalone - Test Data: 0.722
Accuracy for Simplified_Abalone - Training Data: 0.733



Accuracy for Full_Abalone - Test Data: 0.204
Accuracy for Full_Abalone - Training Data: 1.000



(process:21456): GLib-GIO-WARNING **: 08:06:27.039: Unexpectedly, UWP app `Clipchamp.Clipchamp_2.8.3.0_neutral__yxz26nhyzhsrt' (AUMId `Clipchamp.Clipchamp_yxz26nhyzhsrt!App') supports 41 extensions but has no verbs

(process:21456): GLib-GIO-WARNING **: 08:06:27.140: Unexpectedly, UWP app `Microsoft.ScreenSketch_11.2310.54.0_x64__8wekyb3d8bbwe' (AUMId `Microsoft.ScreenSketch_8wekyb3d8bbwe!App') supports 29 extensions but has no verbs


#### (b) Discuss the results seen for the two trees

Simplified Abalone Tree:
Accuracy:
Test Data: 72.2%
Training Data: 73.3%


The tree has decent accuracy on both the training and test sets, suggesting a reasonable generalization to unseen data.
The similarity between training and test accuracies indicates that the model is not overfitting.

Full Abalone Tree:
Accuracy:
Test Data: 20.4%
Training Data: 100%


The model performs well on the training set, achieving 100% accuracy. However, this could be a sign of overfitting.

The significant drop in accuracy on the test set (20.4%) indicates the model doesn't perform well on new, unseen data.


The simplified abalone tree outperforms the full abalone tree in terms of accuracy on both training and test data.
The full abalone tree seems to overfit the training data, resulting in poor performance on the test set.


The full abalone tree is likely more complex, capturing noise in the training data rather than the underlying patterns.
Potential Issues with Full Abalone Tree:

The perfect accuracy on the training set suggests overfitting, where the model memorizes the training data.
The low accuracy on the test set indicates that the model might not generalize well to new data.


Simplified Abalone Tree:

Offers better generalization to new data.
More robust and less likely to be influenced by noise.

Full Abalone Tree:

Demonstrates overfitting to the training data.
Less suitable for making predictions on new, unseen instances.
